In [ ]:
from sqlalchemy import create_engine
from pathlib import Path

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
with engine.connect() as conn, conn.begin():  
    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    # costo_ventas_año_2024 = pd.read_sql_table('costo_ventas_año_2024', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_año = pd.concat([costo_ventas_enero, costo_ventas_febrero, costo_ventas_marzo, costo_ventas_abril, costo_ventas_mayo, costo_ventas_junio, costo_ventas_julio])

In [ ]:
cols_costo_ventas =[
        # Documento de factura
        'name', 'invoice_date', 'partner_name_x', 
        # Vendedora
        'salesperson_id', 'salesperson_name',
        # Producto
        'product_name_x',
        # Precio de la partida
        'quantity', 'price_unit', 'discount', 'price_subtotal',
        # Documento de compra
        'order_name', 'order_date', 'partner_name_y',
        # Costo de la partida
        'product_cost', 'cost_subtotal',
        # Utilidad
        'utilidad_partida_$', 'utilidad_%',
        # Comision clientes
        '%_comision_cliente', 'cost_cliente_comision',
        # Utilidad después de comisión clientes
        'utilidad_partida_$_after_cc',
    ]

In [ ]:
costo_ventas = costo_ventas_julio

## Algo de <span style="color: steelblue">BI</span>

In [ ]:
utilidades = costo_ventas[['salesperson_name', 'price_subtotal', 'cost_subtotal_after_cc', 'utilidad_partida_$_after_cc']].groupby('salesperson_name', dropna=False).sum()

utilidades['utilidad_%_after_cc'] = ((utilidades['price_subtotal'] / (utilidades['cost_subtotal_after_cc']) - 1) * 100).round(2)
utilidades['margen_contribución_%_after_cc'] = (utilidades['utilidad_partida_$_after_cc'] / utilidades['price_subtotal'] * 100).round(2)

utilidades

In [ ]:
utilidades_mes = utilidades.agg({'price_subtotal':['sum'], 'cost_subtotal_after_cc':['sum'], 'utilidad_partida_$_after_cc':['sum']})

utilidades_mes['utilidad_%_after_cc'] = ((utilidades_mes['price_subtotal'] / (utilidades_mes['cost_subtotal_after_cc']) - 1) * 100).round(2)
utilidades_mes['margen_contribución_%_after_cc'] = (utilidades_mes['utilidad_partida_$_after_cc'] / utilidades_mes['price_subtotal'] * 100).round(2)

utilidades_mes